# 이 노트북은 Exploit의 분류 및 Group을 위한 것임

In [19]:
# 저장된 hard/soft 딕셔너리 객체 읽어오기
import pickle

with open("soft_dedup_ngramdf.pkl", 'rb') as f:
    soft_dedup_ngramdf = pickle.load(f)
with open("new_hard_dedup_ngramdf.pkl", 'rb') as f:
    hard_dedup_ngramdf = pickle.load(f)

In [2]:
hard_EID = list()
hard_EID.extend(hard_dedup_ngramdf.EIDs.to_list())
hard_total_eid = list()
for eid in hard_EID:
    hard_total_eid.extend(eid)
len(set(hard_total_eid))

105

In [20]:
soft_EID = list()
soft_EID.extend(soft_dedup_ngramdf.EIDs.to_list())
soft_total_eid = list()
for eid in soft_EID:
    soft_total_eid.extend(eid)
len(set(soft_total_eid))

105

- Hard 중복 제거에서 제거된 익스플로잇 코드는 무엇인가?

In [4]:
set(soft_total_eid) - set(hard_total_eid)

set()

Soft 중복 제거 방법이 온전한 익스플로잇 코드를 보존하고 있기 때문에 Soft 중복 제거 방법을 사용

In [1]:
# EID-CVE Mapping 정보 불러오기
import subprocess
import json

with open("/opt/output/perm/exploit.json", "r") as f:
    exploitJson= json.load(f)

with open("../cve_syscall_mapping/cve_crawling_result.json",encoding='utf-8') as f:
     cveJson = json.load(f)

In [2]:
for e in exploitJson:
    if e['EID'] == "50808":
        print(e)

{'EID': '50808', 'publish': '2022-03-08', 'CVE-ID': '2022-0847', 'src': 'exploitdb', 'url': 'exploit-db.com/exploits/50808', 'CVSS': {'CVSSv2': '7.2', 'CVSSv3': '7.8'}, 'kernel version': '5.8 < 5.16.11'}


In [3]:
# CVE 중복 제거
import pandas as pd
dedup_cve = pd.DataFrame(columns= ["CVEID","CVE_des", "file", "function"])
onecve = {"CVEID":"","CVE_des":"", "file":"", "function":""}
for c in cveJson:
    if not c["CVE-ID"] in dedup_cve.CVEID.to_list():
        onecve["CVEID"] = c["CVE-ID"]
        onecve["CVE_des"] = c["current_description"]
        onecve["file"] = c["file"]
        onecve["function"] = c["function"]
        dedup_cve = dedup_cve.append(onecve, ignore_index=True)
        
dedup_cve

,CVEID,CVE_des,file,function
0,2003-0127,The kernel module loader in Linux kernel 2.2.x...,None,None
1,2003-0961,Integer overflow in the do_brk function for th...,None,do_brk
2,2003-0985,The mremap system call (do_mremap) in Linux ke...,None,None
3,2004-0077,The do_mremap function for the mremap system c...,None,do_munmap
4,2004-0497,Unknown vulnerability in Linux kernel 2.x may ...,None,None
...,...,...,...,...
70,2017-16994,The walk_hugetlb_range function in mm/pagewalk...,mm/pagewalk.c,walk_hugetlb_range
71,2018-14634,An integer overflow flaw was found in the Linu...,None,None
72,2017-11176,The mq_notify function in the Linux kernel thr...,None,mq_notify
73,2019-13272,"In the Linux kernel before 5.1.17, ptrace_link...",kernel/ptrace.c,None


In [16]:
import pandas as pd
allExploitInfo = pd.DataFrame(columns=["EID","CVEID","CVE_des", "file", "function"])
one_exploit = {"EID":"","CVEID":"","CVE_des":"", "file":"", "function":""}
for e in exploitJson:
    if e['EID'] == "50808":
        one_exploit = {"EID":"50808","CVEID":"2022-0847","CVE_des":"A flaw was found in the way the flags member of the new pipe buffer structure was lacking proper initialization in copy_page_to_iter_pipe and push_pipe functions in the Linux kernel and could thus contain stale values. An unprivileged local user could use this flaw to write to pages in the page cache backed by read only files and as such escalate their privileges on the system","file":"fs/splice.c", "function":"None"}
        allExploitInfo = allExploitInfo.append(one_exploit, ignore_index=True)
        one_exploit = {"EID":"","CVEID":"","CVE_des":"", "function":""}
        continue
    for idx, c in dedup_cve.iterrows():
        one_exploit["EID"] = e['EID']
        if c["CVEID"] in e["CVE-ID"]:
            one_exploit["CVEID"] = c["CVEID"]
            one_exploit["CVE_des"] = c["CVE_des"]
            one_exploit["file"] = c["file"]
            one_exploit["function"] = c["function"]
            allExploitInfo = allExploitInfo.append(one_exploit, ignore_index=True)
            one_exploit = {"EID":"","CVEID":"","CVE_des":"", "function":""}
allExploitInfo

,EID,CVEID,CVE_des,file,function
0,3,2003-0127,The kernel module loader in Linux kernel 2.2.x...,None,None
1,12,2003-0127,The kernel module loader in Linux kernel 2.2.x...,None,None
2,131,2003-0961,Integer overflow in the do_brk function for th...,None,do_brk
3,145,2003-0985,The mremap system call (do_mremap) in Linux ke...,None,None
4,160,2004-0077,The do_mremap function for the mremap system c...,None,do_munmap
...,...,...,...,...,...
110,47169,2017-1000112,Linux kernel: Exploitable memory corruption du...,None,None
111,47170,2016-8655,Race condition in net/packet/af_packet.c in th...,net/packet/af_packet.c,None
112,50135,2021-22555,A heap out-of-bounds write affecting Linux sin...,None,None
113,50541,2019-13272,"In the Linux kernel before 5.1.17, ptrace_link...",kernel/ptrace.c,None


In [17]:
import pandas as pd
ExploitCategory = pd.DataFrame(columns = ["EID", "kcomponent", "weak", "CVEID", "CVEinfo"])
one_row = {"EID":"", "kcomponent":"", "weak":"", "CVEID":"", "CVEinfo":""}
for index, row in allExploitInfo.iterrows():
    # file이 있으면
    if not row.file == "None":
        if row.file == "eventpoll.c":
            row.file = "fs"
        one_row["kcomponent"] = row.file
    # file이 없으면
    else:
        print(row.CVEID, row.file)

2003-0127 None
2003-0127 None
2003-0961 None
2003-0985 None
2004-0077 None
2004-0497 None
2004-1235 None
2004-1235 None
2004-1235 None
2005-1294 None
2005-0750 None
2006-2451 None
2006-2451 None
2006-2451 None
2006-2451 None
2007-4573 None
2008-0600 None
2008-0010 None
2008-0009 None
2008-0600 None
2008-0010 None
2008-0009 None
2008-4210 None
2009-1185 None
2009-1527 None
2009-1046 None
2009-2692 None
2009-2692 None
2008-5689 None
2012-0056 None
1999-1166 None
2000-0506 None
2001-0317 None
2001-0317 None
2003-0127 None
2003-0127 None
2005-0750 None
2005-0750 None
2013-1959 None
2013-0268 None
2007-4573 None
2013-6282 None
2009-0343 None
2014-4014 None
2014-4699 None
2012-0056 None
2015-1328 None
2015-3290 None
2015-8660 None
2015-8660 None
2016-0728 None
2016-0728 None
2017-1000112 None
2017-7874 None
2017-7308 None
2017-1000370 None
2017-1000366 None
2017-1000366 None
2017-1000379 None
2017-1000366 None
2017-1000371 None
2017-1000253 None
2017-5123 None
2017-5123 None
2017-16995 None


In [18]:
# 엑셀에 저장
allExploitInfo.to_excel("exploit_info.xlsx")

In [19]:
# pickle로 저장
allExploitInfo.to_pickle("allExploitInfo.pkl")

### allExploitInfo Pickle Load

In [28]:
# 데이터 로드
allExploitInfo = pd.read_pickle('allExploitInfo.pkl')

# 취약점 및 커널 컴포넌트까지 분류 완료된 데이터 프레임 생성

이전에 각 분류 카테고리 정보가 없었던 기존 allExploitInfo에서 수동으로 취약점과 커널 컴포넌트에 대한 분류 후, exploit_category_info.xlsx에 저장했음.

이 데이터를 불러와서 데이터 프레임으로 만들고 정리한 후 다시 pickle로 저장하기

In [20]:
import pandas as pd

allExploitCatInfo = pd.read_excel("exploit_category_info.xlsx", engine="openpyxl")[["EID","Kernel_Component","Vulnerability_Type","CVEID","CVE_des","file","function"]]
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type,CVEID,CVE_des,file,function
0,2013,fs,BA,NaN,NaN,fs/proc,NaN
1,4756,net,BA,NaN,NaN,net/bluetooth,NaN
2,8678,kernel,RC,NaN,NaN,kernel/ptrace.c,NaN
3,10613,fs,ETC,NaN,vmsplice,fs/pipe.c,NaN
4,15023,arch,BA,2010-3301,The IA32 system call emulation functionality i...,arch/x86/ia32/ia32entry.S,None


In [21]:
import numpy as np
allExploitCatInfo = allExploitCatInfo[["EID","Kernel_Component","Vulnerability_Type"]]
# Vulnerability Type 중복인 경우 처리해주기
for index, row in allExploitCatInfo.iterrows():
    if "/" in row.Vulnerability_Type:
        two_vul = row.Vulnerability_Type.split("/")
        row["Vulnerability_Type"]= two_vul[0]
        allExploitCatInfo = allExploitCatInfo.append(row, ignore_index=True)
        row["Vulnerability_Type"] = two_vul[1]
        allExploitCatInfo = allExploitCatInfo.append(row, ignore_index=True)
        allExploitCatInfo["Vulnerability_Type"][index] = np.nan
    if "None" == row.Vulnerability_Type:
        allExploitCatInfo["Vulnerability_Type"][index] = np.nan
allExploitCatInfo = allExploitCatInfo.dropna(axis=0)

/tmp/ipykernel_3871586/3645798574.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allExploitCatInfo["Vulnerability_Type"][index] = np.nan


In [22]:
set(allExploitCatInfo.Vulnerability_Type)

{'AC', 'BA', 'ETC', 'IC', 'IL', 'IV', 'PU', 'RC'}

In [23]:
# EID와 카테고리만 포함된 버전 (pickle)
allExploitCatInfo.to_pickle("onlyCategoryWithEID.pkl")

In [42]:
allExploitInfo["CVEID"][allExploitInfo["EID"]=="5093"].tolist()

['2008-0009']

# 취약점/커널 컴포넌트별 익스플로잇 개수 파악

In [24]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [25]:
kernel_component_set = list(set(allExploitCatInfo.Kernel_Component.to_list()))
vulnerability_set = list(set(allExploitCatInfo.Vulnerability_Type.to_list()))

category = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category.iterrows():
    for c in kernel_component_set:
        category[c][index] = []
for index, row in allExploitCatInfo.iterrows():
    category[row.Kernel_Component][row.Vulnerability_Type].append(row.EID)
    
category.to_excel("attack_category_eid.xlsx")
category.to_pickle("attack_category_eid.pkl")
category.head()

,fs,kernel,net,ipc,security,mm,arch,include,drivers,lib
ETC,[10613],[],[],[],[],[],[],[],[],[]
IC,"[1397, 19419, 25202, 45516, 5092, 5093]","[25444, 26131, 33589, 44298, 45010]","[14814, 41994, 47168]",[],"[39277, 40003]",[],[],[],[9083],[]
AC,"[18411, 2004, 2005, 2006, 2031, 34923, 35161, ...","[38390, 12, 20000, 22362, 22363, 25450, 3, 338...","[15704, 17787, 15916, 15944]",[],[],[],"[27297, 37722, 30604, 4460]",[718],"[15774, 15774]",[]
IL,[],"[44298, 45010]",[17391],[17787],[],[44303],[],[],[],[]
PU,"[33523, 50808, 33321, 33322, 40812, 43345, 50808]",[15704],"[15704, 41458, 9542, 9575, 40871, 47170]",[45553],[],[44303],[],[],[],[]


In [26]:
kernel_component_set = list(set(allExploitCatInfo.Kernel_Component.to_list()))
vulnerability_set = list(set(allExploitCatInfo.Vulnerability_Type.to_list()))

category = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category.iterrows():
    for c in kernel_component_set:
        category[c][index] = []
for index, row in allExploitCatInfo.iterrows():
    category[row.Kernel_Component][row.Vulnerability_Type].append(row.EID)

for index, row in category.iterrows():
    for c in kernel_component_set:
        category[c][index] = len(set(category[c][index]))

category.to_excel("attack_category_cnt.xlsx")
category.to_pickle("attack_category_cnt.pkl")
category

,fs,kernel,net,ipc,security,mm,arch,include,drivers,lib
ETC,1,0,0,0,0,0,0,0,0,0
IC,6,5,3,0,2,0,0,0,1,0
AC,12,12,4,0,0,0,4,1,1,0
IL,0,2,1,1,0,1,0,0,0,0
PU,6,1,6,1,0,1,0,0,0,0
BA,2,0,14,0,0,4,3,1,2,2
IV,3,3,10,0,0,0,1,0,0,0
RC,6,5,4,0,0,2,0,1,1,0
